In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("quickstartod1.csv",sep=",");
df1 = df[['timestep_time', 'vehicle_id', 'vehicle_x', 'vehicle_y']];

df1[1:10]

,timestep_time,vehicle_id,vehicle_x,vehicle_y
1,1,1089,1204.95,5.10
2,1,59,398.35,384.88
3,2,1089,1201.65,15.35
4,2,522,1595.05,394.90
5,2,59,398.35,373.76
6,3,1089,1201.65,25.77
7,3,522,1595.05,384.36
8,3,59,398.35,362.63
9,4,1089,1201.65,36.63


In [2]:
def sample(x, y, x0, y0, r):
    return (x-x0)**2+(y-y0)**2 <= r**2;

In [7]:
sensor1 = [];
sensor1.append([406.55, 190.00]);
dis = 200.00
for i in range(1,8):
    sensor1.append([sensor1[i-1][0]+dis, sensor1[i-1][1]]);
sensor2 = [];
sensor2.append([393.45, 210.00]);
for i in range(1,8):
    sensor2.append([sensor2[i-1][0]+dis, sensor2[i-1][1]]);


In [8]:
for i in range(8):
    df1[`i`+"toRight"] = sample(df1['vehicle_x'], df1['vehicle_y'], sensor1[i][0], sensor1[i][1], 10.00).astype(int);
for i in range(8):
    df1[`i`+"toLeft"] = sample(df1['vehicle_x'], df1['vehicle_y'], sensor2[i][0], sensor2[i][1], 10.00).astype(int);

print df1[1:10]

   timestep_time  vehicle_id  vehicle_x  vehicle_y  0toRight  1toRight  \
1              1        1089    1204.95       5.10         0         0   
2              1          59     398.35     384.88         0         0   
3              2        1089    1201.65      15.35         0         0   
4              2         522    1595.05     394.90         0         0   
5              2          59     398.35     373.76         0         0   
6              3        1089    1201.65      25.77         0         0   
7              3         522    1595.05     384.36         0         0   
8              3          59     398.35     362.63         0         0   
9              4        1089    1201.65      36.63         0         0   

   2toRight  3toRight  4toRight  5toRight  6toRight  7toRight  0toLeft  \
1         0         0         0         0         0         0        0   
2         0         0         0         0         0         0        0   
3         0         0         0      

In [9]:
df1.sort_values(by=['vehicle_id', 'timestep_time'], ascending=True)
sd = [];
for i in range(8):
    sd.append(df1.loc[df1[`i`+'toRight'] == 1])
    sd[i] = sd[i][['timestep_time', 'vehicle_id']]
for i in range(8):
    sd.append(df1.loc[df1[`i`+'toLeft'] == 1])
    sd[i] = sd[i][['timestep_time', 'vehicle_id']]

In [106]:
for i in range(16):
    path = '/Users/Murrie_/Desktop/Artery_With_OD/sensor'+ `i`;
    sd[i].to_csv(path_or_buf = path);